In [2]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
# %cd /content/drive/Shared drives/IML Project/Project
data_path = '/content/drive/Shared drives/IML Project/Project/data/tfrecords_data'
project_path = '/content/drive/Shared drives/IML Project/Project/baseline_LeakyReLu'

In [0]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import metrics

In [0]:
%tensorflow_version 2.x

import tensorflow as tf

from tensorflow.keras import Sequential
from tensorflow.keras.models import load_model

from tensorflow.keras.layers import Input
from tensorflow.keras.layers import Conv3D
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import LeakyReLU
from tensorflow.keras.layers import PReLU
from tensorflow.keras.layers import MaxPool3D
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import AveragePooling3D
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten

In [0]:

batch_size = 32

In [0]:
def decode(serialized_example):
    """
    Parses an image and label from the given `serialized_example`.
    It is used as a map function for `dataset.map`
    """

    features = tf.compat.v1.parse_single_example(
        serialized_example,
        features={
            'img_channels': tf.compat.v1.FixedLenFeature([], tf.int64),
            'img_height': tf.compat.v1.FixedLenFeature([], tf.int64),
            'img_width': tf.compat.v1.FixedLenFeature([], tf.int64),
            'img_raw': tf.compat.v1.FixedLenFeature([], tf.string),
            'sex': tf.compat.v1.FixedLenFeature([], tf.string),
            'age': tf.compat.v1.FixedLenFeature([], tf.float32),
            'label': tf.compat.v1.FixedLenFeature([], tf.string),
        })
    
    
    
    height = tf.cast(features['img_height'], tf.int32)
    width = tf.cast(features['img_width'], tf.int32)
    channels = tf.cast(features['img_channels'], tf.int32)
    image = tf.compat.v1.decode_raw(features['img_raw'], tf.float64)
    image = tf.reshape(image, (channels, height, width, 1))
    label = tf.compat.v1.decode_raw(features['label'], tf.float64)
    sex = tf.cast(features['sex'], tf.string)
    age = tf.cast(features['age'], tf.float32)
    return image, label

In [0]:
train_dataset = tf.data.TFRecordDataset(os.path.join(data_path, 'train.tfrecords'), compression_type='GZIP').map(decode).batch(batch_size)
val_dataset = tf.data.TFRecordDataset(os.path.join(data_path, 'validation.tfrecords'), compression_type='GZIP').map(decode).batch(batch_size)

In [0]:
def create_model():
    kernel_regularizer = tf.keras.regularizers.l2(0.001)
    he_normal = tf.keras.initializers.he_normal(seed=0)
    lecun_normal = tf.keras.initializers.lecun_normal(seed=0)

    input_3d = (62, 96, 96, 1)
    pool_3d = (2, 2, 2)

    model = Sequential()
    
    model.add(tf.keras.layers.InputLayer(input_shape=input_3d))
    model.add(Conv3D(filters=8,
                     kernel_size=3,
                     kernel_regularizer=kernel_regularizer,
                     kernel_initializer=he_normal))
    model.add(BatchNormalization())
    model.add(LeakyReLU())
    model.add(MaxPool3D(pool_size=pool_3d, name='pool1'))

    model.add(Conv3D(filters=8,
                     kernel_size=3,
                     kernel_regularizer=kernel_regularizer,
                     kernel_initializer=he_normal))
    model.add(BatchNormalization())
    model.add(LeakyReLU())
    model.add(MaxPool3D(pool_size=pool_3d, name='pool2'))

    model.add(Conv3D(filters=8,
                     kernel_size=3,
                     kernel_regularizer=kernel_regularizer,
                     kernel_initializer=he_normal))
    model.add(BatchNormalization())
    model.add(LeakyReLU())
    model.add(MaxPool3D(pool_size=pool_3d, name='pool3'))

    model.add(Flatten())
    model.add(Dense(1024, name='dense1', kernel_regularizer=kernel_regularizer, kernel_initializer=he_normal))
    model.add(LeakyReLU())
    model.add(Dropout(0.5, name='dropout1'))

    model.add(Dense(512, activation='relu', name='dense2', kernel_regularizer=kernel_regularizer, kernel_initializer=he_normal))
    model.add(LeakyReLU())
    model.add(Dropout(0.5, name='dropout2'))

    model.add(Dense(3, activation='softmax', name='softmax', kernel_initializer=lecun_normal))

    model.compile(loss='categorical_crossentropy',
                  optimizer=tf.keras.optimizers.Adam(1e-5),
                  metrics=[tf.keras.metrics.CategoricalAccuracy()])

    return model

In [0]:
reducelr_callback = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_categorical_accuracy',
                                                         factor=0.2,
                                                         patience=5,
                                                         min_delta=0.01,
                                                         verbose=1)
checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(os.path.join(project_path, 'checkpoints', 'cp.ckpt'),
                                                         monitor='val_categorical_accuracy',
                                                         verbose=1,
                                                         save_best_only=True,
                                                         mode='max')
weights_callback = tf.keras.callbacks.ModelCheckpoint(os.path.join(project_path, 'weights', 'best_weights.h5'),
                                                      monitor='val_categorical_accuracy',
                                                      verbose=1,
                                                      save_best_only=True,
                                                      save_weights_only=True,
                                                      mode='max')

callbacks_list = [weights_callback, checkpoint_callback, reducelr_callback]

In [23]:
model = create_model()
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_3 (Conv3D)            (None, 60, 94, 94, 8)     224       
_________________________________________________________________
batch_normalization_3 (Batch (None, 60, 94, 94, 8)     32        
_________________________________________________________________
leaky_re_lu_5 (LeakyReLU)    (None, 60, 94, 94, 8)     0         
_________________________________________________________________
pool1 (MaxPooling3D)         (None, 30, 47, 47, 8)     0         
_________________________________________________________________
conv3d_4 (Conv3D)            (None, 28, 45, 45, 8)     1736      
_________________________________________________________________
batch_normalization_4 (Batch (None, 28, 45, 45, 8)     32        
_________________________________________________________________
leaky_re_lu_6 (LeakyReLU)    (None, 28, 45, 45, 8)    

In [0]:
if os.path.exists(os.path.join(project_path, 'checkpoints', 'cp.ckpt')):
    model = load_model(os.path.join(project_path, 'checkpoints', 'cp.ckpt'))

In [0]:
history = model.fit(train_dataset, validation_data=val_dataset, epochs=10, callbacks=callbacks_list, verbose=1)

Epoch 1/10
     66/Unknown - 87s 1s/step - loss: 3.1834 - categorical_accuracy: 0.9175
Epoch 00001: val_categorical_accuracy improved from -inf to 0.75862, saving model to weights/best_weights.h5

Epoch 00001: val_categorical_accuracy improved from -inf to 0.75862, saving model to checkpoints/cp.ckpt
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
INFO:tensorflow:Assets written to: checkpoints/cp.ckpt/assets
66/66 [==============================] - 111s 2s/step - loss: 3.1834 - categorical_accuracy: 0.9175 - val_loss: 3.5632 - val_categorical_accuracy: 0.7586 - lr: 4.0000e-07
Epoch 2/10
66/66 [==============================] - ETA: 0s - loss: 3.1806 - categorical_accuracy: 0.9260
Epoch 00002: val_categorical_accuracy did not improve from 0.75862

Epoch 00002: val_categorical_accuracy did not improve from 0.75862
66/66 [==============================] - 127s 2s/step - loss: 3.1806 - categorical_accuracy: 0.9260 - val_loss: 3.5630 - val_categorical_accura

In [22]:
test_dataset = tf.data.TFRecordDataset(os.path.join(data_path, 'test.tfrecords'), compression_type='GZIP').map(decode).shuffle(100000).batch(batch_size)
results = model.evaluate(test_dataset)

15/15 [==============================] - 16s 1s/step - loss: 3.4720 - categorical_accuracy: 0.7932


In [0]:
def stats2(model):
   
# THis is for modify predicted test labels    
  def cf(x):
      x = np.asarray(x)
      y =[]
      for i in range(len(x)):
          max = np.argmax(x[i])
          y.append(max)
      return np.asarray(y)
  
  def load_test_dataset():
      i = 1
      data_path = '/content/drive/Shared drives/IML Project/Project/data/numpy_data'
      dataset = np.load(os.path.join(data_path, f'img_array_test_6k_{i}.npy'))
      while True:
          try:
              i += 1
              dataset = np.vstack((dataset, np.load(os.path.join(data_path, f'img_array_test_6k_{i}.npy'))))
          except FileNotFoundError:
              print(f'Loaded all test datasets')
              break
      # dataset = np.expand_dims(dataset, axis=1)
      for n in range(dataset.shape[0]):
          dataset[n, :, :] = dataset[n, :, :] / np.amax(dataset[n, :, :].flatten())
      print(f'Normalized {n+1} images')
      dataset = np.reshape(dataset, (-1, 62, 96, 96, 1))
      return dataset, np.eye(3)[labels[labels.train_valid_test == 2]['diagnosis'] - 1]

  labels = pd.read_csv('/content/drive/Shared drives/IML Project/Project/demographics.csv')
  test_data, test_labels = load_test_dataset()

  # Predicted Test labels
  predictions = model.predict(test_data)

  # Modified actual Test labels
  y_test = cf(test_labels)


  # Modified predicted test labels
  y_pred = cf(predictions)

  # Confusion Matrix
  conf = metrics.confusion_matrix(y_test, y_pred)

  accuracy = metrics.accuracy_score(y_test, y_pred)
  f1_score = metrics.f1_score(y_test, y_pred, average = None)
  recall = metrics.recall_score(y_test, y_pred, average= None)

  return accuracy, f1_score, recall

    

    

In [25]:
accuracy, f1_score, recall = stats2(model)
print("The accuracy is " + str(accuracy))
print("F1 Scores are " + str(f1_score))
print("Recall Scores are " + str(recall))

Loaded all test datasets


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:25: RuntimeWarning: invalid value encountered in true_divide


Normalized 29078 images
The accuracy is 0.7931769722814499
F1 Scores are [0.89411765 0.6971831  0.77070064]
Recall Scores are [0.95597484 0.63057325 0.79084967]


In [0]:
# def stats(test_dataset, model):
# # THis is for modify actual test labels
#     def df(t):
#         y =[]
#         for i in range(len(t)):
#             for j in range(len(t[i])):
#                 max = np.argmax(t[i][j])
#                 y.append(max)
#         return np.asarray(y)
    
# # THis is for modify predicted test labels    
#     def cf(x):
#         x = np.asarray(x)
#         y =[]
#         for i in range(len(x)):
#             max = np.argmax(x[i])
#             y.append(max)
#         return np.asarray(y)
    
# # THis is for extract actual test labels    
#     t = []
#     for image, label in test_dataset:
#         t.append(label)

# # Predicted Test labels
#     predictions = model.predict(test_dataset)

# # Modified actual Test labels
#     y_test = df(t)
  
    
# # Modified predicted test labels
#     y_pred = cf(predictions)
    
# # Confusion Matrix
#     conf = metrics.confusion_matrix(y_test, y_pred)

#     accuracy = metrics.accuracy_score(y_test, y_pred)
#     f1_score = metrics.f1_score(y_test, y_pred, average = None)
#     recall = metrics.recall_score(y_test, y_pred, average= None)
    
#     return accuracy, f1_score, recall

    

    